In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

2024-08-12 16:24:26.887807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 16:24:26.908743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 16:24:26.913280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 16:24:26.942997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 16:24:28.249923: W tensorflow/compiler/tf2

In [2]:
print(tf.__version__)


2.17.0


In [3]:
tf.config.list_physical_devices('GPU')

I0000 00:00:1723472669.553130   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472669.627894   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472669.627965   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
import glob
import os
import pathlib

base_dir = pathlib.Path('/home/papafrita/Projects/HAM10k_data/images/')
base_dir

PosixPath('/home/papafrita/Projects/HAM10k_data/images')

In [6]:
image_count = len(list(base_dir.glob('*/*.jpg')))
image_count

10015

In [7]:
benign = list(base_dir.glob('benign/*.jpg'))
print(len(benign))
malignant = list(base_dir.glob('malignant/*.jpg'))
print(len(malignant))


8061
1954


In [8]:
batch_size = 32
img_height = 450
img_width = 600

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size)

Found 10015 files belonging to 2 classes.
Using 8012 files for training.


I0000 00:00:1723472669.943369   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472669.943445   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472669.943473   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472670.270686   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723472670.270770   30110 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-12

In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  base_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 10015 files belonging to 2 classes.
Using 2003 files for validation.


In [11]:
class_names = train_ds.class_names
print(class_names)


['benign', 'malignant']


In [12]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


(32, 450, 600, 3)
(32,)


In [13]:
#Create Model
#    11 layers - Output dense with sigmoid output layer
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


In [14]:
#Compile model

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=[tf.keras.metrics.AUC()]
)


In [15]:
#Use built-in fit function

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=8
)

Epoch 1/8


I0000 00:00:1723472674.233543   30211 service.cc:146] XLA service 0x7f33d40050d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723472674.233737   30211 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2024-08-12 16:24:34.665738: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-12 16:24:34.951596: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  1/251 ━━━━━━━━━━━━━━━━━━━━ 1:04:25 15s/step - auc: 0.7232 - loss: 0.7605

I0000 00:00:1723472687.896674   30211 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


251/251 ━━━━━━━━━━━━━━━━━━━━ 57s 168ms/step - auc: 0.6501 - loss: 0.5190 - val_auc: 0.7584 - val_loss: 0.4149
Epoch 2/8
250/251 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - auc: 0.7474 - loss: 0.4145

...

251/251 ━━━━━━━━━━━━━━━━━━━━ 29s 116ms/step - auc: 0.8292 - loss: 0.3606 - val_auc: 0.8042 - val_loss: 0.3881
Epoch 7/8
250/251 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - auc: 0.8565 - loss: 0.3414

2024-08-12 16:28:39.994108: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134217728 bytes after encountering the first element of size 134217728 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


251/251 ━━━━━━━━━━━━━━━━━━━━ 30s 119ms/step - auc: 0.8566 - loss: 0.3414 - val_auc: 0.8166 - val_loss: 0.3928
Epoch 8/8
250/251 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - auc: 0.8756 - loss: 0.3181

2024-08-12 16:29:10.853209: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134217728 bytes after encountering the first element of size 134217728 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


251/251 ━━━━━━━━━━━━━━━━━━━━ 31s 122ms/step - auc: 0.8757 - loss: 0.3180 - val_auc: 0.8133 - val_loss: 0.4127


In [16]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 450, 600, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 448, 598, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 224, 299, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 222, 297, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 111, 148, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 109, 146, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 54, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 126144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    16,146,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,498,641 (185.01 MB)

 Trainable params: 16,166,213 (61.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 32,332,428 (123.34 MB)

None


In [17]:
#Double check evaluation metrics
metrics = model.evaluate(val_ds)
print("Loss: {}, AUC: {}".format(metrics[0], metrics[1]))

 1/63 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - auc: 0.8704 - loss: 0.3655

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - auc: 0.8054 - loss: 0.4411
Loss: 0.412690669298172, AUC: 0.8133142590522766


In [18]:
#model.save('/home/papafrita/Projects/skin_cancer_tool/Skin_cancer_diagnosis_tool/models/CNN_48M_auc.keras')